# Testing Notebook in Docker.

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("PythonSQL")\
    .getOrCreate()

### Works on Docker

In [11]:
df = spark.read.csv('home_data.csv', header=True)
df.show()
df.printSchema()

+----------+---------------+-------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|        id|           date|  price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|
+----------+---------------+-------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|7129300520|20141013T000000| 221900|       3|        1|       1180|    5650|     1|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|
|6414100192|20141209T000000| 538000|       3|     2.25|       2570|    7242|     2|         0|   0|        3|    7|      2170|          400|    1951

In [20]:
df.filter("price > 1000000").select("price", "sqft_living", "zipcode").show()

+-------+-----------+-------+
|  price|sqft_living|zipcode|
+-------+-----------+-------+
|1225000|       5420|  98053|
|2000000|       3050|  98040|
|1350000|       2753|  98070|
|1325000|       3200|  98004|
|1040000|       4770|  98005|
|1099880|       3520|  98199|
|1088000|       2920|  98004|
|1450000|       2750|  98004|
|2250000|       5180|  98006|
|1095000|       4090|  98033|
|1505000|       3300|  98040|
|1072000|       3900|  98144|
|1025000|       3760|  98033|
|2400000|       3650|  98074|
|2900000|       5050|  98004|
|1365000|       5310|  98077|
|2050000|       3830|  98122|
|3075000|       4550|  98074|
|2384000|       3650|  98119|
|1384000|       4290|  98006|
+-------+-----------+-------+
only showing top 20 rows



### Let's try the registered table

In [32]:
df.createOrReplaceTempView("homes")
expensive_homes_by_zip = spark.sql("select zipcode, count(*) as cnt from homes where price > 1000000 group by zipcode order by 2 desc")
expensive_homes_by_zip.show()

+-------+---+
|zipcode|cnt|
+-------+---+
|  98004|181|
|  98040|135|
|  98112|119|
|  98006|115|
|  98033| 89|
|  98199| 65|
|  98105| 58|
|  98039| 44|
|  98075| 42|
|  98119| 37|
|  98115| 35|
|  98053| 33|
|  98144| 32|
|  98177| 28|
|  98074| 27|
|  98059| 27|
|  98109| 27|
|  98102| 24|
|  98116| 24|
|  98034| 23|
+-------+---+
only showing top 20 rows



## How to change column
Heres how you change the type of the column. Got the code from [here](http://stackoverflow.com/questions/32284620/how-to-change-a-dataframe-column-from-string-type-to-double-type-in-pyspark)

In [34]:
from pyspark.sql.types import DoubleType
test = df.selectExpr("sqft_living as features")
test = test.withColumn("features", test["features"].cast(DoubleType()))

test.printSchema()
test.show()

root
 |-- features: double (nullable = true)

+--------+
|features|
+--------+
|  1180.0|
|  2570.0|
|   770.0|
|  1960.0|
|  1680.0|
|  5420.0|
|  1715.0|
|  1060.0|
|  1780.0|
|  1890.0|
|  3560.0|
|  1160.0|
|  1430.0|
|  1370.0|
|  1810.0|
|  2950.0|
|  1890.0|
|  1600.0|
|  1200.0|
|  1250.0|
+--------+
only showing top 20 rows



### Let's try querying file directly
Notice you have to use the tick around the file path.

TODO: find a way to get specify the header

In [24]:
df_home_file = spark.sql("select * from csv. `home_data.csv`")
df_home_file.show()

+----------+---------------+-------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|       _c0|            _c1|    _c2|     _c3|      _c4|        _c5|     _c6|   _c7|       _c8| _c9|     _c10| _c11|      _c12|         _c13|    _c14|        _c15|   _c16|   _c17|    _c18|         _c19|      _c20|
+----------+---------------+-------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|        id|           date|  price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|
|7129300520|20141013T000000| 221900|       3|        1|       1180|    5650|     1|         0|   0|        3|    7|      1180|            0|    1955